In [1]:
import cv2
import mediapipe as mp
import os
import pandas as pd
import multiprocessing
from multiprocessing import Pool
from pathlib import Path

In [2]:
MODEL = 2

In [3]:
# Initialize MediaPipe components
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [4]:
# Function to process a video file
def process_video(video_path):
    data = []
    with mp_holistic.Holistic(static_image_mode=False, model_complexity=MODEL, 
                              min_detection_confidence=0.4, min_tracking_confidence=0.4) as holistic:
        cap = cv2.VideoCapture(video_path)
        while cap.isOpened():
            ret, image = cap.read()
            if not ret:
                break

            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = holistic.process(image)
            time_ms = cap.get(cv2.CAP_PROP_POS_MSEC)

            # Append pose landmarks data
            if results.pose_landmarks is not None:
                right_shoulder_x = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].x
                right_shoulder_y = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER].y
                left_shoulder_x = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER].x
                left_shoulder_y = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_SHOULDER].y
                right_elbow_x = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].x
                right_elbow_y = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW].y
                left_elbow_x = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW].x
                left_elbow_y = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW].y
                right_wrist_x = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].x
                right_wrist_y = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST].y
                left_wrist_x = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].x
                left_wrist_y = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_WRIST].y
                right_eye_x = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_EYE].x
                right_eye_y = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_EYE].y
                left_eye_x = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EYE].x
                left_eye_y = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EYE].y
                nose_x = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.NOSE].x
                nose_y = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.NOSE].y
                right_index_x = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_INDEX].x
                right_index_y = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_INDEX].y

                data.append([time_ms, right_shoulder_x, right_shoulder_y, left_shoulder_x, left_shoulder_y, 
                             right_elbow_x, right_elbow_y, left_elbow_x, left_elbow_y, 
                             right_wrist_x, right_wrist_y, left_wrist_x, left_wrist_y, 
                             right_eye_x, right_eye_y, left_eye_x, left_eye_y, right_index_x, right_index_y,
                             nose_x, nose_y])


        cap.release()
    return data



In [5]:
# Worker function to process a single video
def process_video_file(video_path):
    video_data = process_video(video_path)
    df = pd.DataFrame(video_data, columns=[
    "time_ms", 
    "right_shoulder_x", "right_shoulder_y", 
    "left_shoulder_x", "left_shoulder_y", 
    "right_elbow_x", "right_elbow_y", 
    "left_elbow_x", "left_elbow_y", 
    "right_wrist_x", "right_wrist_y", 
    "left_wrist_x", "left_wrist_y", 
    "right_eye_x", "right_eye_y", 
    "left_eye_x", "left_eye_y",
    "nose_x", "nose_y",
    "right_index_x", "right_index_y" 
    ])
    csv_file_name = "/n/kfranich_speech_l3/Lab/6000_EMA/processed_articulatory_data/video_processing/keypoints/SN6004_ARTGEST_ENG_VIDAUD/" + os.path.splitext(os.path.basename(video_path))[0] + "_keypoints.csv"
    df.to_csv(csv_file_name, index=False)
    return f"DataFrame saved as {csv_file_name}"

# Main script to process all MOV files in a directory using multiprocessing
def main(directory_path):
    video_files = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.endswith(".MOV")]

    with Pool() as pool:
        results = pool.map(process_video_file, video_files)

    for result in results:
        print(result)

if __name__ == "__main__":
    directory_path = "/n/kfranich_speech_l3/Lab/6000_EMA/video_and_audio_data/SN6004_ARTGEST_ENG_VIDAUD/SN6004_ARTGEST_ENG_VIDAUD_GEST"
    main(directory_path)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


DataFrame saved as /n/kfranich_speech_l3/Lab/6000_EMA/processed_articulatory_data/video_processing/keypoints/SN6004_ARTGEST_ENG_VIDAUD/SN6004_ARTGEST_ENG_G_VCV_IN_1_keypoints.csv
DataFrame saved as /n/kfranich_speech_l3/Lab/6000_EMA/processed_articulatory_data/video_processing/keypoints/SN6004_ARTGEST_ENG_VIDAUD/SN6004_ARTGEST_ENG_G_CVCV_IN_keypoints.csv
DataFrame saved as /n/kfranich_speech_l3/Lab/6000_EMA/processed_articulatory_data/video_processing/keypoints/SN6004_ARTGEST_ENG_VIDAUD/SN6004_ARTGEST_ENG_G_VCV_IN_2_keypoints.csv
DataFrame saved as /n/kfranich_speech_l3/Lab/6000_EMA/processed_articulatory_data/video_processing/keypoints/SN6004_ARTGEST_ENG_VIDAUD/SN6004_ARTGEST_ENG_G_CVC_keypoints.csv
DataFrame saved as /n/kfranich_speech_l3/Lab/6000_EMA/processed_articulatory_data/video_processing/keypoints/SN6004_ARTGEST_ENG_VIDAUD/SN6004_ARTGEST_ENG_G_VCV_FIN_keypoints.csv
DataFrame saved as /n/kfranich_speech_l3/Lab/6000_EMA/processed_articulatory_data/video_processing/keypoints/SN6